В дополнение к реализованному нами методу рассмотрим реализацию из библиотеки `scipy.optimize`. Для замеров её эффективности и построения графиков будем использовать ScipyWrapper написанный в прошлой лабораторной.

In [ ]:
import sys
sys.path.insert(0, "./lib")
from lrs import *
from gradient_descent import ScipyWrapper
from newton import NewtonCG
from function_wrapper import FunctionWrapper
from output import pretty_print
from graphics_plotter import GraphicsPlotter

Для начала рассмотрим простую функцию и сравним как работает на ней библиотечный метод, а как реализованный нами.
$(x - 2)^2 + (y - 4)^2 + 1$

In [ ]:
func = lambda: FunctionWrapper(lambda x: (x[0] - 2)**2 + (x[1] - 4)**2 + 1)
bounds = [[-5, 5], [-5, 5]]
start = [-2, 2]
max_iter = 1000

gradient.clear()
hessian.clear()
newton = NewtonCG(constant(1), func(), bounds)
res = newton.find_min(start, max_iter)
pretty_print(newton, "NEWTON", res, gradient, hessian) 

gradient.clear()
hessian.clear()
f = func()
wrapper = ScipyWrapper(f, bounds)
res = wrapper.find_min("Newton-CG", start, max_iter, lambda x: gradient(f, x, 1e-5), lambda x: hessian(f, x, 1e-5))
pretty_print(wrapper, "Scipy", res, gradient, hessian)

plotter1 = GraphicsPlotter(newton)
plotter1.plot("Our Newton")
plotter2 = GraphicsPlotter(wrapper)
plotter2.plot("Scipy-Newton")

Теперь посмотрим на интересную функцию из 1го задания "Банановая долина". $(1 - x)^2 + 100(y-x^2)^2$

In [ ]:
func = lambda: FunctionWrapper(lambda x: (1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2)

bounds = [[-5, 5], [-5, 5]]
start = [-2, 2]
max_iter = 1000

gradient.clear()
hessian.clear()
f = func()
wrapper = ScipyWrapper(f, bounds)
res = wrapper.find_min("Newton-CG", start, max_iter, lambda x: gradient(f, x, 1e-5), lambda x: hessian(f, x, 1e-5))
pretty_print(wrapper, "Scipy", res, gradient, hessian)

plotter2 = GraphicsPlotter(wrapper)
plotter2.plot("Scipy-Newton")

In [ ]:
for c in [1, 0.5, 0.1]:
    gradient.clear()
    hessian.clear()
    newton = NewtonCG(constant(c), func(), bounds)
    plotter1 = GraphicsPlotter(newton)
    res = newton.find_min(start, max_iter)
    pretty_print(newton, "NEWTON", res, gradient, hessian) 
    plotter1.plot("Our Newton")

По данным графикам отлично видно, что наш метод повторяет библиотечную реализацию, кроме выбора шага. В зависимости от выбранной длины шага наш метод делает разное кол-во шагов и лишь при шаге длиной 0.1 от изначальной, мы приблизились по кол-ву итераций к библиотечной реализации. При этом на предыдущем примере методы отработали примерно одинаково - это наводит на мысль, что библиотечная реализация использует динамический метод выбора шага, мы с ними тоже поэксперементировали в 1 задании, но ничего лучше 1 на функциях, выбранных нами, найти не удалось.

Теперь посмотрим на реализованный в библиотеке метод "BFGS". Его преимущество в том, что он не вычисляет обратный Гессиан, а пересчитывает его итеративно, получая ассимптотику $O(n^2)$ на итерацию. Начнём также с простой квадратичной функции.

In [ ]:
func = lambda: FunctionWrapper(lambda x: (x[0] - 2)**2 + (x[1] - 4)**2 + 1)
bounds = [[-5, 5], [-5, 5]]
start = [-2, 2]
max_iter = 1000

gradient.clear()
hessian.clear()
f = func()
wrapper = ScipyWrapper(f, bounds)
res = wrapper.find_min("BFGS", start, max_iter, lambda x: gradient(f, x, 1e-5))
pretty_print(wrapper, "Scipy", res, gradient)

plotter2 = GraphicsPlotter(wrapper)
plotter2.plot("Scipy-BFGS")

Как видно, он тоже отлично доходит до минимальной точки, но гораздо меньше раз вызвал функцию при том же кол-ве шагов т.к. не считал Гессианы. Теперь посмотрим на банановую долину, на которой библиотечная реализация метода Ньютона работала достаточно долго.

In [ ]:
func = lambda: FunctionWrapper(lambda x: (1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2)

bounds = [[-5, 5], [-5, 5]]
start = [-2, 2]
max_iter = 1000

gradient.clear()
hessian.clear()
f = func()
wrapper = ScipyWrapper(f, bounds)
res = wrapper.find_min("BFGS", start, max_iter, lambda x: gradient(f, x, 1e-5))
pretty_print(wrapper, "Scipy", res, gradient)

plotter2 = GraphicsPlotter(wrapper)
plotter2.plot("Scipy-BFGS")

А вот здесь уже отличия разительные. BFGS - сделал в 4 раза меньше шагов, соответственно гораздо меньше раз посчитал функцию, не говоря о том, что каждая итерация была в 2 раза быстрее чем у метода Ньютона. Давайте посмотрим как ведёт себя данный метод на мультимодальных функциях (вдруг он лишён и этого недостатка метода Ньютона) 

In [ ]:
func_sin = lambda c: FunctionWrapper(lambda x: np.sin(c * x[0]) + (x[0] + x[1])**2 + x[0]**2 + x[1]**2)

bounds = [[-2.5, 2.5], [-2.5, 2.5]]
start = [-2, -2]
max_iter = 1000

gradient.clear()
hessian.clear()
f = func_sin(5)
wrapper = ScipyWrapper(f, bounds)
res = wrapper.find_min("BFGS", start, max_iter, lambda x: gradient(f, x, 1e-5))
pretty_print(wrapper, "Scipy", res, gradient)

plotter2 = GraphicsPlotter(wrapper)
plotter2.plot("Scipy-BFGS")

In [ ]:
plotter2.plot_3d("Scipy-BFGS")

Вывод: BFGS - из библиотеки scipy.optimize - это улучшенная версия метода Ньютона. Она лишена одного из недостатков - работает кратно быстрее, однако под капотом делает тоже самое, что и метод Ньютона - ищет точку, в которой градиент равен 0 и поэтому всё ещё не работает на мультимодальных функциях.